In [2]:

#Import
import requests
import pandas as pd
from datetime import datetime, date
from dotenv import load_dotenv
import os


In [44]:
#Chargement des variables d'environnement
load_dotenv()
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
DB_INTERVENTIONS_ID = os.getenv("DB_INTERVENTIONS_ID")
DB_INVOICES_ID = os.getenv("DB_INVOICES_ID")

In [45]:
HEADERS = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

In [6]:
#Conversion date format FR
def convert_date_fr_to_iso(date_str: str) -> str:
    return datetime.strptime(date_str, "%d/%m/%Y").strftime("%Y-%m-%d")


In [8]:
#Récupération des données Notion
def get_database_properties(database_id):
    url = f"https://api.notion.com/v1/databases/{database_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    return response.json()

props = get_database_properties(DB_INTERVENTIONS_ID)

In [ ]:
#Récupération des données Notion avec filtres : ETAPE 2
def query_unbilled_entries(date_begin: str, date_end: str, a_ete_facture: bool):

    """
    Récupère les interventions dont la case 'A facturer' est cochée ou non,
    et qui ont une date comprise entre date_begin et date_end.
    """
    date_begin_fr = convert_date_fr_to_iso(date_begin)
    date_end_fr = convert_date_fr_to_iso(date_end)
    
    query = {
        "filter": {
            "and": [
                {
                    "property": "Facturé",
                    "checkbox": {"equals": a_ete_facture}
                },
                {
                    "property": "Date de début",
                    "date": {"on_or_after": date_begin_fr}
                },
                {
                    "property": "Date de fin",
                    "date": {"on_or_before": date_end_fr}
                }
            ]
        }
    }

    if a_ete_facture is None :
        query = {
            "filter": {
                "and": [
                    {
                        "property": "Date de début",
                        "date": {"on_or_after": date_begin_fr}
                    },
                    {
                        "property": "Date de fin",
                        "date": {"on_or_before": date_end_fr}
                    }
                ]
            }
        }

    response = requests.post(
        f"https://api.notion.com/v1/databases/{DB_INTERVENTIONS_ID}/query",
        headers=HEADERS,
        json=query
    )

    if not response.ok:
        print("Erreur API Notion :", response.status_code)
        print(response.text)

    response.raise_for_status()

    return response.json()["results"]


In [17]:
def parse_interventions_to_dataframe(results):
    rows = []
    for page in results:
        props = page["properties"]

        try:
            ville = props.get("Ville", {}).get("select", {}).get("name", "Inconnu")
            ecole = props.get("Ecole", {}).get("select", {}).get("name", "Inconnue")
            classe = props.get("Classe", {}).get("select", {}).get("name", "Inconnue")
            heures = float(props.get("Nombre d’heures", {}).get("number", 0))
            tarif = float(props.get("Tarif horaire", {}).get("number", 0))
            total = heures * tarif
            date_debut_str = props.get("Date de début", {}).get("date", {}).get("start")
            date_debut = pd.to_datetime(date_debut_str) if date_debut_str else None

            rows.append({
                "Ville": ville,
                "Ecole": ecole,
                "Classe": classe,
                "Heures": heures,
                "Tarif horaire": tarif,
                "Total": total,
                "Date de début": date_debut
            })

        except Exception as e:
            print("Erreur parsing d'une ligne:", e)
            continue

    return pd.DataFrame(rows)

In [18]:
#Affichage des résultats avec dataframe
date_begin_fr = "01/06/2025"
date_end_fr = "30/06/2025"
resultats = query_unbilled_entries(date_begin_fr, date_end_fr, a_ete_facture=False)
df = parse_interventions_to_dataframe(resultats)
display(df.head())


,Ville,Ecole,Classe,Heures,Tarif horaire,Total,Date de début
0,Noisy-le-Grand,ESIEE,M1,0.0,5.0,0.0,2025-06-10


In [ ]:
#ETAPE 3: Création facture

def create_invoice_page(client: str, interventions: list, total: float, mois: str, invoice_number: str) -> dict:
    children = []

    #Ajouter chaque ligne de prestation
    for item in interventions:
        props = item["properties"]
        try:
            cours = props["Cours"]["title"][0]["text"]["content"]
            heures = float(props["Nombre d’heures"]["number"])
            tarif = float(props["Tarif horaire"]["number"])
            total_ligne = heures * tarif

            ligne = f"{cours} | {heures}h | {tarif}€ | {total_ligne}€"
        except Exception as e:
            print("❌ Erreur de lecture d’une intervention :", e)
            continue

        children.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {"type": "text", "text": {"content": ligne}}
                ]
            }
        })

    # Ajout total
    children.append({
        "object": "block",
        "type": "callout",
        "callout": {
            "icon": {"type": "emoji", "emoji": "💰"},
            "rich_text": [
                {"type": "text", "text": {"content": f"Total à payer : {total} €"}}
            ]
        }
    })

    payload = {
        "parent": {"database_id": DB_INVOICES_ID},
        "properties": {
            "Client": {
                "title": [{"text": {"content": client}}]
            },
            "Mois": {
                "rich_text": [{"text": {"content": mois}}]
            },
            "Total Amount": {
                "number": total
            },
            "Invoice Number": {
                "rich_text": [{"text": {"content": invoice_number }}] 
            }
        },
        "children": children
    }

    response = requests.post(
        "https://api.notion.com/v1/pages",
        headers=HEADERS,
        json=payload
    )
    print("❌ Erreur API Notion :", response.status_code)
    print(response.text) 

    response.raise_for_status()
    return response.json()


In [ ]:
client = "TEST"
mois = datetime.now().strftime("%Y-%m")
invoice_number = "RONI"

resultats = query_unbilled_entries(date_begin_fr, date_end_fr, a_ete_facture=False)

#Filtrage
interventions_client = [
    item for item in resultats
    if item["properties"]["Ecole"]["select"]["name"] == client
]

#Calcul du total
total = sum(
    i["properties"]["Nombre d’heures"]["number"] * i["properties"]["Tarif horaire"]["number"]
    for i in interventions_client
)

interventions_client = interventions_client[:10] #Evite le gateway timeout de Notion
#Création de la facture dans Notion
response = create_invoice_page(client, interventions_client, total, mois, invoice_number)

print("Facture créée avec l’ID :", response["id"])

❌ Erreur API Notion : 200
{"object":"page","id":"2112cdbb-4757-81da-a157-e94250039f9a","created_time":"2025-06-13T09:14:00.000Z","last_edited_time":"2025-06-13T09:14:00.000Z","created_by":{"object":"user","id":"3ad9a6c5-500e-4a6b-addf-a7e3896a77d6"},"last_edited_by":{"object":"user","id":"3ad9a6c5-500e-4a6b-addf-a7e3896a77d6"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"2102cdbb-4757-8110-a76f-cb2284e29df3"},"archived":false,"in_trash":false,"properties":{"Mois":{"id":"Jh%5EE","type":"rich_text","rich_text":[{"type":"text","text":{"content":"2025-06","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"2025-06","href":null}]},"Invoice Number":{"id":"NoqM","type":"rich_text","rich_text":[{"type":"text","text":{"content":"RONI","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"RONI",

In [47]:
props = get_database_properties(DB_INVOICES_ID)
from pprint import pprint
pprint(props["properties"]["Invoice Number"])


{'id': 'NoqM', 'name': 'Invoice Number', 'rich_text': {}, 'type': 'rich_text'}
